# Lab2 : Recognizing strings of CFG using ANTLR

In this lab, we will familiarize ourselves with building CFG driven parsers using ANTLR.

## The language

Recall the language of _balanced_ brackets is *not* regular, but can be expressed using CFG.

Let's define a language involving only one type of token: `WORD`.

A word is a string consisting of only characters from `a-z` and `A-Z`.  The token can be defined by the lexer rule:

```
WORD : ('a' .. 'z' | 'A' .. 'Z')+;
```

We will ignore whitespaces with the following lexer rule.

```
WHITESPACE : (' ' | '\t' | '\r' | '\n')+ -> skip;
```

With the tokens defined, we can define a syntactic variable called `phrase`, and two alternations.  A phrase is a WORD but with
zero or more balanced parentheses.

```
phrase : WORD
       | '(' phrase ')'
       ;
```

Finally, we will have a start symbol:

```
startSymbol : phrase;
```

# Your Tasks

In this lab, you will cover all the essential steps to author an ANTLR grammar.

1. Author the lexer and parser rules in an ANTLR `mygrammar/My.g4` grammar file.
    Make sure your grammar file contains the following header section:
    ```
    @header {
    package mygrammar;
    }
    ```


2. Compile Java `.class` files from the ANTLR grammar file.  Refer to the `Makefile` provide.


3. Use Kotlin to access the generated grammar files.

Load the dependencies.

**Remember**: the kernel must be restarted whenver the `mygrammar` classes are recompiled.

In [ ]:
@file:DependsOn("/data/shared/antlr-4.9.1-complete.jar")
@file:DependsOn(".")

Import the necessary classes.

In [ ]:
import org.antlr.v4.runtime.*
import mygrammar.*

We will use our own error handler to be more vocal about parsing errors.

In [ ]:
val errorlistener = object: BaseErrorListener() {
    override fun syntaxError(recognizer: Recognizer<*,*>,
           offendingSymbol: Any?,
           line: Int,
           pos: Int,
           msg: String,
           e: RecognitionException?) {
        throw Exception("${e} at line:${line}, char:${pos}")
    }
}

Perform parsing:

1. Construct lexer and parser.
2. Use the custom error listener.
3. Return the start symbol 

In [ ]:
fun parse(s: String): MyParser.StartSymbolContext? {
    //
    // Construct lexer and parser.
    //
    val input = CharStreams.fromString(s)
    val lexer = MyLexer(input)
    val tokens = CommonTokenStream(lexer)
    val parser = MyParser(tokens)
    
    // Use our own error listener so it reports
    // any parsing error
    lexer.removeErrorListeners()
    lexer.addErrorListener(errorlistener)
    parser.removeErrorListeners()
    parser.addErrorListener(errorlistener)

    // Perform parsing, and return the context.
    try {
        return parser.startSymbol()
    } catch(e: Exception) {
        println("Parse Error: " + e.message)
        return null
    }
}

We can easily build a function that recognizes strings in the language.

In [ ]:
fun recognize(s: String) = parse(s) != null

# Test Cases

One can convince oneself that `(hello)` should belong to the language.

In [ ]:
recognize("(hello)")

In fact any level of balanced parentheses should be accepted: like `(((world)))`

In [ ]:
recognize("(((world)))")

But `(hello` should be rejected.

In [ ]:
recognize("(hello")

Since only a WORD can exist between parenthese, `(123)` is also rejected.

In [ ]:
recognize("(123)")

We should also reject `(hello world)` because we only allow one word between the parenthese.

In [ ]:
recognize("(hello world)")

Clearly `(hello))` should be rejected.  If you are not careful, you will find that this gets accepted.

If you fail this test, make sure you terminate your `startSymbol` with an EOF token.

In [ ]:
recognize("(hello))")